In [4]:
"""
Working on a big refactor here...
moving towards vrAnalysis2

Objectives:
- Build dataclass structure for imaging and behavior sessions that I can operate on
across datatypes - I want to run the same code on my data and external data
- Simplify analyses by building a set of functions that can be called on these dataclasses
instead of extending analysis classes for each kind of analysis
- Rewrite standard scripts to do analyses with this new refactor


I should think about the key requirements for analyzing behavior and imaging data for
vr navigation and structure around that. Things that I do:

Basic data handling:
- Load spiking data
- Load behavior data (position, speed, etc)
- Load ROI data

Processing:
- Make a spkmap
- Process spkmaps
- Filter and sort ROIs by tracked (or other criteria, like plane_idx...)
"""


"""
Ideas:
An analysis object which can be loaded that isn't necessarily connect to any data. You can pass in a list
of data objects to it and it will run the analysis on all of them. This way I can run the same analysis on
my data and external data, compare within a mouse, across mice, etc.

There can be a "flag" for same mice with tracker, in which case it can perform special operations like aligning
ROIs across sessions by tracked ROI etc. 
"""


print()

In [2]:
%reload_ext autoreload
%autoreload 2

from vrAnalysis2.external.pettit2022 import find_pettit_harvey_sessions, data_path
sessions = find_pettit_harvey_sessions(data_path / "dataFolder")

behavior = sessions[0].behavior
spks = sessions[0].spks

In [ ]:
from pathlib import Path
from vrAnalysis2 import files
from vrAnalysis import database
sessiondb = database.vrDatabase('vrSessions')
from typing import Union

def find_experiment_options(root_dir: Union[str, Path]) -> list[Path]:
    """
    Find all vrExperimentOptions.json files in the given directory and its subdirectories.
    
    Parameters
    ----------
    root_dir : str or Path
        The root directory to start the search from
    
    Returns
    -------
    list[Path]
        List of paths to all matching files
    """
    def make_identifier(pth: Path) -> list[str]:
        return "_".join(list(reversed([p.stem for p in list(pth.parents)[:3]])))
    
    root_path = Path(root_dir)
    all_paths = list(root_path.rglob("vrExperimentOptions.json"))
    session_identifier = [make_identifier(pth) for pth in all_paths]
    return all_paths, session_identifier

pths, sids = find_experiment_options(files.local_data_path())
csesids = [sessiondb.sessionPrint(joinby="_") for sessiondb in sessiondb.iterSessions(useDefault=True)]

for sid in sids:
    if sid not in csesids:
        print("oops", sid)

for ses in sessiondb.iterSessions():
    csesid = ses.sessionPrint(joinby="_")
    print(csesid, csesid in sids)